In [1]:
import pandas as pd
import numpy as np

In [2]:
%run limpieza.ipynb
df_eventos_up_to_01062018 = get_clean_df()

In [3]:
df_features = pd.DataFrame(index=df_eventos_up_to_01062018['person'].unique())
df_features.index.name = 'person'

In [4]:
df_features.shape

(38829, 0)

In [5]:
df_features.head()

""
person
4886f805
ad93850f
0297fc1e
2d681dd8
cccea85e


# Features por device

In [6]:
events_per_device_count = df_eventos_up_to_01062018.groupby('person')['device_type'].value_counts()
events_per_device_count.head()

person    device_type
0008ed71  Computer        2
00091926  Computer       34
00091a7a  Smartphone      1
000ba417  Computer        6
000c79fe  Smartphone      1
Name: device_type, dtype: int64

In [7]:
events_per_device_count= events_per_device_count.unstack().fillna(0)
events_per_device_count.head(10)

device_type,Computer,Smartphone,Tablet
person,,,
0008ed71,2.0,0.0,0.0
00091926,34.0,0.0,0.0
00091a7a,0.0,1.0,0.0
000ba417,6.0,0.0,0.0
000c79fe,0.0,1.0,0.0
000e4d9e,13.0,0.0,0.0
000e619d,5.0,0.0,0.0
001001be,0.0,3.0,0.0
0010e89a,1.0,0.0,0.0


In [8]:
df_features = df_features.merge(events_per_device_count, how='inner', on='person')

In [9]:
df_features.head()

,Computer,Smartphone,Tablet
person,,,
4886f805,0.0,1.0,0.0
ad93850f,0.0,5.0,0.0
0297fc1e,0.0,95.0,0.0
2d681dd8,2.0,0.0,0.0
cccea85e,22.0,0.0,0.0


# Feature vive en Brasil

In [10]:
df_eventos_up_to_01062018['vive en Brasil'] = df_eventos_up_to_01062018['country']=='Brazil'
df_vive_Brasil = df_eventos_up_to_01062018.loc[df_eventos_up_to_01062018['event']=='visited site',['person','vive en Brasil']]

In [11]:
df_vive_Brasil = df_vive_Brasil.drop_duplicates()

In [12]:
df_features = df_features.merge(df_vive_Brasil, how='inner', on='person')

In [13]:
df_features.shape

(38788, 5)

In [14]:
df_features.head()

,person,Computer,Smartphone,Tablet,vive en Brasil
0,4886f805,0.0,1.0,0.0,True
1,ad93850f,0.0,5.0,0.0,True
2,0297fc1e,0.0,95.0,0.0,True
3,2d681dd8,2.0,0.0,0.0,True
4,cccea85e,22.0,0.0,0.0,True
